# On main branch, this file should be non executed and no parameters set.
# On dev_ branch, it must be fully executed and must include tests.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import sqlite3

## Functions

In [3]:
def calc_averages(columnLabel, results_df):
    '''
    Calculate the means for grouped values of columnLabel.
    
    Parameters:
    columnLabel should be in ['mT', 'm2T', 'm4T'].
    results_df contains the temporal averages.
    
    Returns a pandas series of replications and networks averages.
    '''
    
    # Do the mean over the replications
    x = pd.DataFrame(results_df.groupby(['v','N','q','net'])[columnLabel].mean())
    x.reset_index(inplace=True)
    
    # Do the mean over the network configuration
    x = x.groupby(['v','N','q'])[columnLabel].mean()
    
    
    return x

## Parameters

In [4]:
# Set the filename of the raw results database.
# By raw I mean coming directly from simulations.
#
# Example: 'bubble_filtering__8_neighbors_squared_network.db'

db_filename = 'bubble_filtering__8_neighbors_squared_network.db'

## Main
At the normal uses, the below code should not be changed.

In [5]:
# Read the database with pandas

conn = sqlite3.connect('../results_databases/' + db_filename)

df = pd.read_sql('SELECT * FROM temporal_mean_results', conn,
                 index_col = 'index'
                )
df.sort_values(['v','N','q','net'], inplace=True)

conn.close()

In [6]:
df.head(20)

,v,N,q,net,rep,mT,m2T,m4T,mT_noMod,std_,std_noMod
index,,,,,,,,,,,
0,0.25,400,0.0000,-1,1,0.206236,0.063894,0.010397,-0.001027,0.146153,0.252770
1,0.25,400,0.0005,-1,1,0.201640,0.061150,0.009585,-0.004404,0.143147,0.247245
2,0.25,400,0.0010,-1,1,0.204047,0.062297,0.009877,0.008375,0.143741,0.249452
3,0.25,400,0.0015,-1,1,0.201144,0.060938,0.009605,-0.001429,0.143104,0.246852
4,0.25,400,0.0020,-1,1,0.199880,0.059965,0.009207,-0.005571,0.141466,0.244814
5,0.25,400,0.0025,-1,1,0.196470,0.058164,0.008743,0.001456,0.139871,0.241168
6,0.25,400,0.0030,-1,1,0.194409,0.057042,0.008468,-0.006340,0.138735,0.238751
7,0.25,400,0.0035,-1,1,0.192952,0.056545,0.008431,0.009466,0.138975,0.237602
8,0.25,400,0.0040,-1,1,0.191659,0.055435,0.008037,-0.001032,0.136757,0.235445


Let be $m$ the sequence of values of the magnetization along the time for an evolution of the system.

$M = <<<m>_{T}>_{R}>_{C}$

The temporal mean $<m>_{T}$ is already calculated at raw results.

In [8]:
# Calculate the physical interesting quantities

phys_quant = pd.DataFrame(calc_averages('mT', df))
phys_quant.reset_index(inplace=True)
phys_quant.rename({'mT': 'M'},axis=1,inplace=True)

m2df = calc_averages('m2T', df)
m4df = calc_averages('m4T', df)

phys_quant['M_err'] = np.array(m2df) - phys_quant['M']**2
phys_quant['M_noMod'] = np.array(calc_averages('mT_noMod', df))
phys_quant['M_noMod_err'] = np.array(m2df) - phys_quant['M_noMod']**2

phys_quant['X'] = phys_quant['N']*phys_quant['M_err']
phys_quant['X_noMod'] = phys_quant['N']*phys_quant['M_noMod_err']

phys_quant['U'] = 1 - np.array(m4df)/(3*np.array(m2df)**2)

phys_quant['u'] = -np.log(1-3*phys_quant['U']/2)

In [9]:
phys_quant.head(20)

,v,N,q,M,M_err,M_noMod,M_noMod_err,X,X_noMod,U,u
0,0.25,400,0.0000,0.206236,0.021361,-0.001027,0.063893,8.544282,25.557141,0.151106,0.257036
1,0.25,400,0.0005,0.201640,0.020491,-0.004404,0.061130,8.196461,24.452135,0.145517,0.246253
2,0.25,400,0.0010,0.204047,0.020661,0.008375,0.062226,8.264586,24.890545,0.151625,0.258043
3,0.25,400,0.0015,0.201144,0.020479,-0.001429,0.060936,8.191512,24.374318,0.137813,0.231578
4,0.25,400,0.0020,0.199880,0.020013,-0.005571,0.059934,8.005042,23.973501,0.146494,0.248130
5,0.25,400,0.0025,0.196470,0.019564,0.001456,0.058162,7.825518,23.264818,0.138524,0.232923
6,0.25,400,0.0030,0.194409,0.019247,-0.006340,0.057002,7.698948,22.800780,0.132533,0.221645
7,0.25,400,0.0035,0.192952,0.019314,0.009466,0.056455,7.725626,22.581965,0.121037,0.200350
8,0.25,400,0.0040,0.191659,0.018702,-0.001032,0.055434,7.480947,22.173759,0.128244,0.213646
9,0.25,400,0.0045,0.188344,0.018535,0.013792,0.053818,7.413896,21.527200,0.119207,0.197001


In [10]:
# Persist the dataframe
raise Exception('Do not execute this cell accidentally.')

phys_quant.to_csv(
    '../results_databases/' + db_filename[:-3] + '__MXU.csv')

<br><br>
## Testing

In [ ]:
# The test below is just a check, once I know some plot for the 
# physical quantities I am doing just a visual inspection.

In [ ]:
phys_quant.head()

In [ ]:
x = phys_quant[phys_quant.v == 1]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()

In [ ]:
x = phys_quant[phys_quant.v == 0.4]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()

In [ ]:
x = phys_quant[phys_quant.v == 0.25]
N = x.N.unique()
x.set_index(['N','q'], inplace=True)
for n in N:
    x.loc[n].M.plot(label=n)
plt.title('Magnetization (absolute value before temporal avg.')
plt.legend(loc=(1.05,0))
plt.show()

for n in N:
    x.loc[n].u.plot(label=n)
plt.title('Binder cumulant')
plt.legend(loc=(1.05,0))
plt.show()